Counter mode only provides confidentiality and changing one byte by an attacker only changes one byte of plaintext, unlike Cipher block chaining mode

In [1]:
# This demonstrates how someone could change a message if the knew the plain text and position, but not the key
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# Key and nonce (IV for CTR mode) generation
key = os.urandom(32)
nonce = os.urandom(16)

# Create a plaintext message
message = "This is a secret message. Don't let anyone see it."

# Create a cipher object
aesCipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())

# Encryption
aesEncryptor = aesCipher.encryptor()
ciphertext = aesEncryptor.update(message.encode()) + aesEncryptor.finalize()

# At this point, suppose an attacker knows that "Don't let anyone see it." is part of the plaintext.
# They want to change that to "Attack at dawn."

known_plaintext = "Don't let anyone see it.".encode()
new_plaintext =   "Attack at dawn.".encode()
known_ciphertext = ciphertext[-len(known_plaintext):]  # Assuming the attacker knows where in the ciphertext this part is

# The zip() function in Python returns an iterator of tuples, where the i-th tuple contains the i-th element from each
# of the input iterables. The iterator stops when the shortest input iterable is exhausted.

# So if encrypted_counter and new_plaintext are not of the same length, the zip() function will stop at the end
# of the shortest one. This means the new_ciphertext will be truncated to the length of the shortest input iterable.

# This scenario could happen if the new_plaintext you're trying to substitute is longer than the known_plaintext.
# In the provided code example, it's assumed that the new_plaintext and the known_plaintext are of the same length
# to avoid such a situation.

# In a more generalized scenario where you're trying to substitute a new_plaintext that's longer than the known_plaintext,
# you'll face problems because AES-CTR uses a unique counter value for each block of plaintext. If your new plaintext 
# spans more blocks than the known plaintext, you won't have the correct counter values for the additional blocks. To 
# successfully substitute a longer plaintext, you'd need to know the entire plaintext and ciphertext, or at least 
# enough of it to determine the counter values for all blocks you want to modify.

# The attacker can XOR the known ciphertext with the known plaintext to recover the encrypted counter
encrypted_counter = bytes(a ^ b for a, b in zip(known_ciphertext, known_plaintext))

# Then XOR that with the new plaintext to produce a new ciphertext
new_ciphertext = bytes(a ^ b for a, b in zip(encrypted_counter, new_plaintext))

# The attacker replaces the old ciphertext with the new one in the message
modified_ciphertext = ciphertext[:-len(known_plaintext)] + new_ciphertext

# Decryption
aesDecryptor = aesCipher.decryptor()
decryptedtext = aesDecryptor.update(modified_ciphertext) + aesDecryptor.finalize()

print("Decrypted message: ", decryptedtext.decode())

Decrypted message:  This is a secret message. Attack at dawn.


In [1]:
# This demonstrates how someone could determine where multiple messages are the same if the same IV and key is used for them. 
# This allows an attacker to know what part of the plaintext to focus on capturing so they could modify future messages or get the keys
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# Key and nonce (IV for CTR mode) generation
key = os.urandom(32)
nonce = os.urandom(16)

# Create a plaintext message
message1 = "This is the header.ABCDEFGHIJKLMNOPQRSTUVWXYZ"
message2 = "This is the header.BCDEFGHIJKLMNOPQRSTUVWXYZA"

# Create a cipher object
aesCipher1 = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())
aesCipher2 = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())

# Encryption
aesEncryptor1 = aesCipher1.encryptor()
ciphertext1 = aesEncryptor1.update(message1.encode()) + aesEncryptor1.finalize()

aesEncryptor2 = aesCipher2.encryptor()
ciphertext2 = aesEncryptor2.update(message2.encode()) + aesEncryptor2.finalize()

# The attacker can look for where the message is unchanged this is because
# c1 = m1 ^ K
# c2 = m2 ^ K
# c1 ^ c2 = (m1 ^ K) ^ (m2 ^ K)
# c1 ^ c2 = (m1 ^ m2) ^ (K ^ K)
# c1 ^ c2 = (m1 ^ m2) # This will be zero where the message is the same
c1_xor_c2_message = bytes(a ^ b for a, b in zip(ciphertext1, ciphertext2))

is_fixed_text = [ 'Y' if c1_xor_c2 == 0 else '.'  for c1_xor_c2 in c1_xor_c2_message]
print(f"is_fixed_text: {''.join(is_fixed_text)}")
print(f"    message 1: {message1}")
print(f"    message 2: {message2}")

is_fixed_text: YYYYYYYYYYYYYYYYYYY..........................
    message 1: This is the header.ABCDEFGHIJKLMNOPQRSTUVWXYZ
    message 2: This is the header.BCDEFGHIJKLMNOPQRSTUVWXYZA


### Overview
AES CCM (Counter with CBC-MAC) and AES CTR (Counter) are both modes of operation for the AES (Advanced Encryption Standard) algorithm, but they serve different purposes and have different characteristics. Here's a breakdown of how they differ:

### AES CTR (Counter Mode)
- Encryption Only: AES CTR mode provides encryption but does not provide any integrity verification or authentication. It simply encrypts data.
- Counter as Nonce: It uses a counter value that is incremented for each block of plaintext. The counter, along with a nonce (number used once), is used to generate the block cipher input.
- Parallelizable: Encryption and decryption processes are parallelizable because each block is encrypted independently of the others.
- Efficiency: It can operate efficiently on large streams of data and is suitable for environments where only confidentiality is required.

### AES CCM (Counter with CBC-MAC)
- Encryption and Authentication: AES CCM mode provides both encryption and data authentication. It combines CTR mode for encryption with CBC-MAC (Cipher Block Chaining Message Authentication Code) for authentication.
- Non-Parallelizable Authentication: The authentication step (CBC-MAC) is not parallelizable because it processes each block of data sequentially, depending on the output of the previous block.
- Nonce and Additional Data: It uses a nonce (which must not be repeated with the same key) and can incorporate additional authenticated data (AAD) into the authentication process without encrypting it, providing data integrity and authenticity for both the encrypted payload and additional plaintext data.
- Overhead: CCM mode has some overhead due to the authentication tag, which must be sent along with the ciphertext to verify the integrity and authenticity of the data at the receiver's end.
- Use Cases: It is widely used in applications where both data integrity and confidentiality are important, such as in wireless communication standards like IEEE 802.11i (Wi-Fi) and 802.15.4 (used in Thread and Zigbee).

### Summary
The primary difference between AES CTR and AES CCM is that CTR provides only confidentiality, whereas CCM provides confidentiality, integrity, and authenticity. This makes CCM a more comprehensive solution for security, but it introduces more computational overhead and complexity compared to CTR. These characteristics determine their suitability for different applications based on security requirements.

In [2]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers.aead import AESCCM

def encrypt_message(message, key, nonce):
    # Ensure the key and nonce lengths are suitable for AES-CCM
    aesccm = AESCCM(key)
    ciphertext = aesccm.encrypt(nonce, message.encode(), None)
    return ciphertext

def decrypt_message(ciphertext, key, nonce):
    try:
        aesccm = AESCCM(key)
        plaintext = aesccm.decrypt(nonce, ciphertext, None)
        return plaintext.decode()
    except Exception as e:
        print("Decryption failed:", e)
        return None

# Key and nonce (IV for CCM mode) generation
key = os.urandom(32)  # AES-256 requires a 32-byte key
nonce = os.urandom(13)  # CCM mode recommends a 13-byte nonce for 802.15.4

# Create a plaintext message
message = "This is a secret message. Don't let anyone see it."

# Encryption
ciphertext = encrypt_message(message, key, nonce)
print("Encrypted message:", ciphertext)

# Decryption
decrypted_message = decrypt_message(ciphertext, key, nonce)
print("Decrypted message:", decrypted_message)


Encrypted message: b'\xf1\xe3\x1a\xb9@\xe2\x12Pw\x99\x9aX\r\x120\xe3>\x97\x03Q\xec\xa5\xba.\xc2\x9f\x80\x19]\xa5]\xe3,\\\xb9q\x9d\xc3\x89\xae_\xc0{\x1f&:\x02\xc3\xc7\x82\xa4\x01\x19\xff\xce\xa7\x139\xd8\xbbe\xd2\xa3\xec\xf6V'
Decrypted message: This is a secret message. Don't let anyone see it.
